<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Analyzing Customer Sentiment using ClearScape Analytics
  <br>
       <img id="teradata-logo" src="../../images/TeradataLogo.png" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>
<hr>

<br>

<b style = 'font-size:24px;font-family:Arial;color:#00233C'>Utilize native Vantage capabilities for Text Analytics and performance at extreme scale</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Sentiment Analysis is defined as the process of computationally identifying and categorizing opinions expressed in a piece of text, especially in order to determine whether the writer's attitude towards a particular topic, product, etc. is positive, negative, or neutral.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Many sentiment analysis tools exist, but most of them cannot perform flexibly enough and at a scale that allows the ability to deploy these insights to <b>all</b> types of use cases including real-time, high concurrency, or extreme volume</p> 

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Teradata Vantage native Natural Langyage Analysis functions, combined with whole-data set statistical analysis; can provide deep insights into natural language intent, patterns, and author sentiment at extreme scale, speed, and concurrency.</p> 
<hr>

<b style = 'font-size:24px;font-family:Arial;color:#00233C'>Live Demonstration</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following demonstration will illustrate how simple it is to utilize these native functions to understand customer sentiment, and measure the accuracy of machine-generated predictions.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Additionally, since Vantage supports client tools and applications ranging from the most modern programmatic APIs such as Python, REST, and R; to traditional dashboards, BI tools, and analytics via SQL, a virtually unlimited number and types of users and applications can consume the results of this deep analysis; effectively democratizing specialized analytics to the widest possible audience.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The data for this demonstration consists of Amazon Fine Foods Reviews, which can be found <a href = 'https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews'>here</a>.  The Demonstration below consists of three steps:</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Analyze the raw data set, including customer rating values</li>
    <li>Execute native Sentiment Analysis function and inspect the output</li>
    <li>Analyze the results and compare to the author rating</li>
    </ol>
    
<img src = 'Flow_Diagram_Sentiment.png' width = 800>

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Step 1 - Analyze the original data set</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we will inspect the original data set, and perform some analysis on customer-provided ratings values.</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Inspect the rows of the table</li>
    <li>Analyze the distribution of ratings</li>
    <li>Visualize the results</li>
    </ol>
    
<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Imports and Connection</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Import required packages and create a connection context to Vantage.</p>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import json
from teradataml import *
display.suppress_vantage_runtime_warnings = True

from IPython.display import display as ipydisplay

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# load vars json
with open('../../vars.json', 'r') as f:
    session_vars = json.load(f)

# Create the SQLAlchemy Context
host = session_vars['environment']['host']
username = session_vars['hierarchy']['users']['business_users'][1]['username']
password = session_vars['hierarchy']['users']['business_users'][1]['password']

eng = create_context(host=host, username=username, password=password)

eng.execute(f'''SET SESSION COMPUTE GROUP {session_vars['hierarchy']['users']['business_users'][1]['compute_group']}''')

# confirm connection
print(eng)

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>1.1 - Inspect the Data</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a "Virtual Dataframe" which is a remote representation of the data set.  This allows us to operate on the data at remove/at scale using common pandas and python syntax.</p>

In [ ]:
tdf_reviews = DataFrame('"demo_ofs"."Amazon_Fine_Foods_Reviews"')

In [ ]:
ipydisplay(tdf_reviews.shape)
ipydisplay(tdf_reviews.sample(2))

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>1.2 Analyze the distribution of author ratings</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a simple aggregation query to calculate distribution</p>

In [ ]:
df = tdf_reviews.groupby('rating').count().to_pandas()
ipydisplay(df)
df.set_index('rating', drop = True)[['count_review']].sort_index().plot(kind = 'bar');

<hr>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Step 2- Use Sentiment Extractor to calculate the sentiment score</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Steps are as follow:</p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Call the SentimentExtractor function on reviews data</li>
    <li>Inspect the Results</li>
    <li>Create a new table that can be used for analysis later</li>
    </ol>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The SentimentExtractor function uses a dictionary model to extract the sentiment (positive, negative, or neutral) of each input document or sentence.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The function handles negated sentiments as follows:</p>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>-1 if the sentiment is negated (for example, "I am not happy")</li>
    <li>-1 if the sentiment and a negation word are separated by one word (for example, "I am not very happy")</li>
    <li>+1 if the sentiment and a negation word are separated by two or more words (for example, "I am not saying I am happy")</li>
    </ul>
<hr>   

In [ ]:
sentiment_res = SentimentExtractor(data = tdf_reviews, text_column = 'review', accumulate = ['review', 'rating'])

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2.1 SentimentExtractor Output</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The function returns an object containing a new dataframe with new columns (in addition to the columns declared in the Accumulate parameter):</p>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>content</b>  The column contains the sentence extracted from the document. The column displays if you use Sentence as the AnalysisType.</li>
    <li><b>polarity</b>  The sentiment value of the result. Possible values are POS (positive), NEG (negative), or NEU (neutral)</li>
    <li><b>sentiment_score</b>  The sentiment score of polarity. Possible values are 0 (neutral), 1 (higher than neutral), or 2 (higher than 1)</li>
    <li><b>sentiment_words</b>  The string that contains a total positive score, total negative score, and sentiment words with their polarity_strength and frequency enclosed in parenthesis.</li>
    </ul>

In [ ]:
sentiment_res.result

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Step 3 - Analyze the results and compare to author rating</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Additional analytic and data processing functions can be used to provide deeper insights into model accuracy, or inform better tuning of the analytic process.  What follows are some examples:</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Create a sentiment polarity out of the author ratings</li>
    <li>Use Vantage Functions to analyze the classification accuracy of our model</li>
    <li>Create a ratings value out of sentiment score and polarity</li>
    <li>Calculate the distributions of ratings values and visualize results</li>
 </ol>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.1 Create a categorical representation of author ratings.</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>For purposes of comparison, assume the ratings values can be categorized into Negative, Neutral, and Positive categories to match the output of the Sentiment Analysis function.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we will use the <b>BinCode</b> ClearScape Analytics Function, which provides the ability to create custom-width, and custom-labeled categorical labels from a column of numeric values.  This function takes the following arguments:</p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>data</b>. The teradataml DataFrame that points to the data to be bin-coded.</li>
    <li><b>target_columns</b>.  The column or columns to apply the bin-coding</li>
    <li><b>method_type</b>.  Variable or fixed-width.  "Variable" then takes a set of custom bin directives:</li>
    <li><b>fit_data</b>.  A "fit" table reference can be passed that contains specific directives for custom binning.  This fit table contains the target column(s) to apply the binning, min and max bin values, and a target label for the resulting bin.  Additional arguments address the binning, such as min value column, max value column, label column, etc.</li>
</ul>

In [ ]:
bincode_fit = BincodeFit(data = sentiment_res.result, 
                 target_columns = 'rating',
                 method_type = 'Variable-Width',
                 fit_data = DataFrame('"demo"."bin_table"'),
                 target_colnames = 'ColumnName',
                 minvalue_column = 'MinValue',
                 maxvalue_column = 'MaxValue',
                 label_column = 'Label')

tdf_transformed = BincodeTransform(data = sentiment_res.result, 
                                   object = bincode_fit.output, 
                                   accumulate = ['sentiment_score','polarity'])
tdf_transformed.result

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.2 Using ClassificationEvaluator to check accuracy</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In classification problems, a confusion matrix is used to visualize the performance of a classifier. The confusion matrix contains predicted labels represented across the row-axis and actual labels represented across the column-axis. Each cell in the confusion matrix corresponds to the count of occurrences of labels in the test data.  See the <a href = 'https://docs.teradata.com/r/Teradata-VantageTM-Analytics-Database-Analytic-Functions-17.20/Advanced-SQL-Engine-Analytic-Functions/TD_ClassificationEvaluator'>documentation</a> for more details.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This function takes an input table (in this case, the dataframe we created above) and function parameters including the columns that ate to be compared.</p>

In [ ]:
class_eval = ClassificationEvaluator(data = tdf_transformed.result, 
                                     observation_column = 'rating', 
                                     prediction_column = 'polarity',
                                     labels = ['NEG','NEU','POS'])

In [ ]:
ipydisplay(class_eval.result)
ipydisplay(class_eval.output_data)

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.3 Visualize the results</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Use open-source tools to create a heatmap.</p>

In [ ]:
df_pred = tdf_transformed.result.to_pandas(all_rows = True)
cm = confusion_matrix(df_pred['rating'], df_pred['polarity'])
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = ['NEG', 'NEU', 'POS'])
fig, ax = plt.subplots(figsize=(10,10))
disp.plot(ax=ax)

plt.show()

In [ ]:
remove_context()